In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
 8192/11490434 [..............................] - ETA: 0s 1753088/11490434 [===>..........................] - ETA: 0s 9953280/11490434 [========================>.....] - ETA: 0s11493376/11490434 [==============================] - 0s 0us/step

## Visualizing the Image Data

In [5]:
x_train.shape

Out[6]: (60000, 28, 28)

In [6]:
single_image = x_train[0]

In [7]:
single_image.shape

Out[8]: (28, 28)

In [8]:
plt.imshow(single_image)

# PreProcessing Data

We first need to make sure the labels will be understandable by our CNN.

## Labels

In [11]:
y_train

Out[10]: array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [12]:
y_test

Out[11]: array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Hmmm, looks like our labels are literally categories of numbers. We need to translate this to be "one hot encoded" so our CNN can understand, otherwise it will think this is some sort of regression problem on a continuous axis. Luckily , Keras has an easy to use function for this:

In [14]:
from tensorflow.keras.utils import to_categorical

In [15]:
y_train.shape

Out[13]: (60000,)

In [16]:
y_example = to_categorical(y_train)

In [17]:
y_example

Out[15]: array([[0., 0., 0., ..., 0., 0., 0.],
 [1., 0., 0., ..., 0., 0., 0.],
 [0., 0., 0., ..., 0., 0., 0.],
 ...,
 [0., 0., 0., ..., 0., 0., 0.],
 [0., 0., 0., ..., 0., 0., 0.],
 [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [18]:
y_example.shape

Out[16]: (60000, 10)

In [19]:
y_example[0]

Out[17]: array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [20]:
y_cat_test = to_categorical(y_test,10)

In [21]:
y_cat_train = to_categorical(y_train,10)

### Processing X Data

We should normalize the X data

In [23]:
single_image.max()

Out[20]: 255

In [24]:
single_image.min()

Out[21]: 0

In [25]:
x_train = x_train/255
x_test = x_test/255

In [26]:
scaled_single = x_train[0]

In [27]:
scaled_single.shape

Out[24]: (28, 28)

In [28]:
scaled_single.max()

Out[25]: 1.0

In [29]:
plt.imshow(scaled_single)

## Reshaping the Data

Right now our data is 60,000 images stored in 28 by 28 pixel array formation. 

This is correct for a CNN, but we need to add one more dimension to show we're dealing with 1 RGB channel (since technically the images are in black and white, only showing values from 0-255 on a single channel), an color image would have 3 dimensions.

In [31]:
x_train.shape

Out[27]: (60000, 28, 28)

In [32]:
x_test.shape

Out[28]: (10000, 28, 28)

Reshape to include channel dimension (in this case, 1 channel)

In [34]:
x_train = x_train.reshape(60000, 28, 28, 1)

In [35]:
x_train.shape

Out[30]: (60000, 28, 28, 1)

In [36]:
x_test = x_test.reshape(10000,28,28,1)

In [37]:
x_test.shape

Out[32]: (10000, 28, 28, 1)

## Training the Model

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Conv2D, 
                                     MaxPool2D, Flatten)

In [40]:
model = Sequential()

# CONVOLUTIONAL LAYER()
model.add(Conv2D(filters=32, kernel_size=(4,4),
                 input_shape=(28,28,1), activation='relu'))
# POOLING LAYER
model.add(MaxPool2D(pool_size=(2,2)))

# FLATTEN IMAGES FROM 28 by 28 to 764 BEFORE FINAL LAYER
model.add(Flatten())

# 128 NEURONS IN DENSE HIDDEN LAYER (WE CAN CHANGE THIS NUMBER OF NEURONS)
model.add(Dense(units=128, activation='relu'))

# LAST LAYER IS THE CLASSIFIER, THUS 10 POSSIBLE CLASSES
model.add(Dense(10, activation='softmax'))

# https://keras.io/metrics/
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy']) # we can add in additional metrics

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d (Conv2D) (None, 25, 25, 32) 544 
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32) 0 
_________________________________________________________________
flatten (Flatten) (None, 4608) 0 
_________________________________________________________________
dense (Dense) (None, 128) 589952 
_________________________________________________________________
dense_1 (Dense) (None, 10) 1290 
=================================================================
Total params: 591,786
Trainable params: 591,786
Non-trainable params: 0
_________________________________________________________________

In [42]:
from tensorflow.keras.callbacks import EarlyStopping

In [43]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

## Train the Model

In [45]:
model.fit(x_train,y_cat_train,
         validation_data=(x_test,y_cat_test),
         epochs=10,
         verbose=2,
         callbacks=[early_stop])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 - 27s - loss: 0.1375 - acc: 0.9590 - val_loss: 0.0514 - val_acc: 0.9838
Epoch 2/10
60000/60000 - 27s - loss: 0.0472 - acc: 0.9856 - val_loss: 0.0432 - val_acc: 0.9861
Epoch 3/10
60000/60000 - 27s - loss: 0.0312 - acc: 0.9907 - val_loss: 0.0352 - val_acc: 0.9887
Epoch 4/10
60000/60000 - 27s - loss: 0.0207 - acc: 0.9937 - val_loss: 0.0439 - val_acc: 0.9868
Epoch 5/10
60000/60000 - 28s - loss: 0.0151 - acc: 0.9950 - val_loss: 0.0409 - val_acc: 0.9876
Out[40]: <tensorflow.python.keras.callbacks.History at 0x7f1cc359feb8>

In [46]:
metrics = pd.DataFrame(model.history.history)

In [47]:
metrics

,loss,acc,val_loss,val_acc
0,0.137474,0.958967,0.051356,0.9838
1,0.047196,0.985633,0.043206,0.9861
2,0.031191,0.990700,0.035191,0.9887
3,0.020693,0.993650,0.043879,0.9868
4,0.015137,0.994983,0.040872,0.9876


In [48]:
metrics[['loss','val_loss']].plot()

In [49]:
metrics[['acc','val_acc']].plot()